In [5]:
import os,sys
sys.path.insert(0,'../../libs')
from utils import load_json

### Load all API keys 
openai_key = load_json('/home/chuang/Dev/Keys/openai_key.json') 
hf_key = load_json('/home/chuang/Dev/Keys/huggingface_key.json')
os.environ['OPENAI_API_KEY'] = openai_key['ChatGPT']['API_KEY']
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hf_key['HuggingFace']['API_KEY']

#### Prompt Template

In [36]:
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [9]:
### create a more factual based QA system ### 
### fist give instruction 
### then give context 
### then user question 
### then answer 
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """
prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

We can pass the prompt directly to openai 

In [13]:
# initialize the models
openai = OpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key=os.environ['OPENAI_API_KEY'],
    temperature=0.1,
)

In [16]:
print(openai(
    prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
))

Hugging Face, OpenAI, and Cohere offer LLMs.


Example of changing answer style with few shot in context learning 

In [28]:
prompt = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 

User: How are you?
AI: I can't complain but sometimes I still do.

User: What time is it?
AI: It's time to get a watch.

User: What is the meaning of life?
AI: """

print(openai(prompt))

I'm not sure, but I think it involves a lot of pizza and Netflix.


An clearner way of doing this 

In [37]:
# create a long examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "How about you tell me?"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)
# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

##################################################
## we can also use length based example selector##
# ################################################ 
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100  # this sets the max length that examples should be
) 
# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)




In [39]:
query = "What is the meaning of life?"
print(openai(few_shot_prompt_template.format(query=query)))

I already told you, it's 42. Did you forget already?


In [38]:
query = "What is the meaning of life?"
print(openai(dynamic_prompt_template.format(query=query)))

42. But seriously, it's up to you to find your own meaning and purpose in life.


In [42]:
##################################################
## a better version is similarity based selector ##
# ################################################ 
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [43]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples, 
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(), 
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma, 
    # This is the number of examples to produce.
    k=3
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [44]:
print(similar_prompt.format(query=query))
query = "What is the meaning of life?"
print(openai(similar_prompt.format(query=query)))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: What is the meaning of life?
AI: 42


User: Who is your best friend?
AI: Siri. We have spirited debates about the meaning of life.


User: What is your favorite movie?
AI: Terminator


User: What is the meaning of life?
AI: 
42, but don't ask me to explain it. I'm just a humble AI assistant, not a philosopher.
